# Homework #3 - Rossman Store Sales

Group Members:

Hoo De Lin (A0126576X)

Yap Han Chiang (A0125168E)

# Analyzing the data:

Before we try figuring out which model to fit in, we need to understand the data, and clean/modify the data to be able to feed into any model

The feature that I believe are quite important are:

- Competition
- Holidays
- Type of stores
- Type of day (Monday - Sunday)
- Having promotions
- Promo2 since

Interesting to note is that, 

### Combining HasHoliday, isTomorrowHoliday and isYesterdayHoliday into holidayBeforeAfter has just slightly better results ###

### 66 days of training data ###
2014-11-26 to 2015-01-30
### 30 days of testing data ###
2015-07-02 to 2015-07-31

In [464]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn import svm
import matplotlib.pyplot as pl
%matplotlib inline
import seaborn as sb 
import csv
from sklearn.ensemble import RandomForestRegressor

# import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error

In [465]:
trainData = pd.read_csv('train_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str'})
storeData = pd.read_csv('store.csv')
testData = pd.read_csv('test_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str', 'StoreType': 'str', 'Assortment': 'str'})
trainData = pd.merge(trainData, storeData, on='Store')
testData = testData.reset_index().merge(storeData,on='Store')
index = testData['index']
testData.drop('index', axis=1, inplace=True)

In [18]:
trainData

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-01-30,5577,616,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-01-29,5171,573,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-01-28,4310,537,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-01-27,4806,532,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-01-26,4781,566,1,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
5,1,7,2015-01-25,0,0,0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
6,1,6,2015-01-24,4840,600,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
7,1,5,2015-01-23,3586,489,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
8,1,4,2015-01-22,3492,493,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
9,1,3,2015-01-21,3299,460,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [466]:
mapCharToInt = {'0': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4}
month_name_num_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7,
                        'Aug': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
month_num_name_dict = {
    "1": "Jan",
    "2": "Feb",
    "3": "Mar",
    "4": "Apr",
    "5": "May",
    "6": "Jun",
    "7": "Jul",
    "8": "Aug",
    "9": "Sept",
    "10": "Oct",
    "11": "Nov",
    "12": "Dec"
}

daysInMonth = {0:0, 1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10:31, 11: 30, 12:31}

In [467]:
# Cleaning the data
competition_distance_mean = trainData['CompetitionDistance'].mean()
customer_mean = trainData['Customers'].mean()
competition_distance_std = trainData['CompetitionDistance'].std()
customer_std = trainData['Customers'].std()

def normalizeDistance(row):
    return (row['CompetitionDistance'] - competition_distance_mean) / competition_distance_std
def normalizeCustomer(row):
    return (row['Customers'] - customer_mean) / customer_std
def hasHoliday(row):
    if str(row['StateHoliday']) != "0" or str(row['SchoolHoliday']) != "0":
        return 1
    else:
        return 0
def calculateCompetitionAge(row):
        if math.isnan(row['CompetitionOpenSinceMonth']) or int(row['CompetitionOpenSinceMonth']) == 0:
            return 0
        else:
            f_date = datetime(int(row['CompetitionOpenSinceYear']), int(row['CompetitionOpenSinceMonth']), 1, 0, 0, 0)
            return math.floor(((row['Date'].to_pydatetime() - f_date).days) / 30)

def invert(row):
    if int(row['CompetitionDistance']) == 0:
        return 0
    else:
        return 1 / math.sqrt(row['CompetitionDistance'])

def isHavingPromo2(row):
    if not isinstance(row['PromoInterval'], str):
        return 0
    elif month_num_name_dict[str(row['Date'].to_pydatetime().month)] in row['PromoInterval']:
        return 1
    else:
        return 0
    
# Possible thought: If we are not at the promo interval, should promo2Age have any effect?
def promo2Age(row):
    if row['Promo2'] == 0:
        return 0
    else:
#        https://stackoverflow.com/a/17087427
        date = str(int(row["Promo2SinceYear"])) + "-W" + str(int(row["Promo2SinceWeek"]))
        f_date = datetime.strptime(date + '-0', "%Y-W%W-%w")
        return math.floor((row['Date'].to_pydatetime() - f_date).days/ 30)

def getDayOfMonth(row):
    return str(row['Date']).split('-')[2].split(' ')[0]

def getMonth(row):
    return row['Date'].month

def isTomorrowHoliday(row):
    current = row['isTomorrowHoliday']
    if (row['Date'].to_pydatetime() + timedelta(1) == row['previousRowDate']):
        return current
    else:
        return 0
def isYesterdayHoliday(row):
    current = row['isYesterdayHoliday']
    if (row['Date'].to_pydatetime() - timedelta(1) == row['nextRowDate']):
        return current
    else:
        return 0   

In [468]:
trainData.fillna(0, inplace=True)
testData.fillna(0, inplace=True)

In [390]:
trainData.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-01-30,5577,616,1,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,4,2015-01-29,5171,573,1,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1,3,2015-01-28,4310,537,1,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,1,2,2015-01-27,4806,532,1,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,1,1,2015-01-26,4781,566,1,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


In [469]:
trainData['HasHoliday'] = trainData.apply(lambda row: hasHoliday(row), axis=1)
trainData['previousRowDate'] = trainData['Date'].shift(1)
trainData['nextRowDate'] = trainData['Date'].shift(-1)
trainData['isTomorrowHoliday'] = trainData['HasHoliday'].shift(1)
trainData['isYesterdayHoliday'] = trainData['HasHoliday'].shift(-1)
trainData['isTomorrowHoliday'] = trainData.apply(lambda row: isTomorrowHoliday(row), axis=1)
trainData['isYesterdayHoliday'] = trainData.apply(lambda row: isYesterdayHoliday(row), axis=1)
trainData.drop(['previousRowDate','nextRowDate'],inplace=True, axis=1)
trainData['CompetitionAge'] = trainData.apply(lambda row: calculateCompetitionAge(row), axis=1)
trainData['HavingPromo2'] = trainData.apply(lambda row: isHavingPromo2(row), axis=1)
trainData['Promo2Age'] = trainData.apply(lambda row: promo2Age(row), axis=1)
trainData['CompetitionDistance'] = trainData.apply(lambda row: invert(row), axis=1)
trainData['DayOfMonth'] = trainData.apply(lambda row: getDayOfMonth(row), axis=1)
trainData['Month'] = trainData.apply(lambda row: getMonth(row), axis=1)
trainData.replace(mapCharToInt, inplace=True)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f14fda17a58>>
Traceback (most recent call last):
  File "/home/han/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


In [470]:
testData['HasHoliday'] = testData.apply(lambda row: hasHoliday(row), axis=1)
testData['previousRowDate'] = testData['Date'].shift(1)
testData['nextRowDate'] = testData['Date'].shift(-1)
testData['isTomorrowHoliday'] = testData['HasHoliday'].shift(1)
testData['isYesterdayHoliday'] = testData['HasHoliday'].shift(-1)
testData['isTomorrowHoliday'] = testData.apply(lambda row: isTomorrowHoliday(row), axis=1)
testData['isYesterdayHoliday'] = testData.apply(lambda row: isYesterdayHoliday(row), axis=1)
testData.drop(['previousRowDate','nextRowDate'],inplace=True, axis=1)
testData['CompetitionAge'] = testData.apply(lambda row: calculateCompetitionAge(row), axis=1)
testData['HavingPromo2'] = testData.apply(lambda row: isHavingPromo2(row), axis=1)
testData['Promo2Age'] = testData.apply(lambda row: promo2Age(row), axis=1)
testData['CompetitionDistance'] = testData.apply(lambda row: invert(row), axis=1)
testData['DayOfMonth'] = testData.apply(lambda row: getDayOfMonth(row), axis=1)
testData['Month'] = testData.apply(lambda row: getMonth(row), axis=1)
testData.replace(mapCharToInt, inplace=True)

## Trying out some features... ##

### startPromo2. Does it help: Very slightly? ###

In [471]:
def isStartPromo2(row):
    if int(row['Promo2']) == 0:
        return 0
    else:
        #
        day = row['Date'].day
        if day != 1:
            return 0
        else:
            month = row['Date'].month
            for promoMonth in row['PromoInterval'].split(','):
                if (month_num_name_dict[str(month)] == promoMonth):
                    if day == 1:
                        return 1
            return 0

In [472]:
trainData['isStartPromo2'] = trainData.apply(lambda row: isStartPromo2(row), axis=1)
testData['isStartPromo2'] = testData.apply(lambda row: isStartPromo2(row), axis=1)

### weekly sales average. Does it help: No, it sucks###

In [473]:
def weeklySalesAverage():
    stopAt = 21
    weeklySales = 0
    count = 0
    for i in range(trainData.shape[0]):
        weeklySales += trainData.loc[i].Sales
        count += 1
        if count == 7:
            weekAverage = weeklySales / 7
            count = 0
            # print("Week {} sales: {}".format((i+1)/7, weeklySales))
            # print("Week {} average sales: {}".format((i+1)/7, weekAverage))
            weeklySales = 0
            for j in range(i-6, i+1):
                trainData.set_value(j, 'weeklyAverageSales', weekAverage)
    if count > 0:
        for i in range(trainData.shape[0]-count, trainData.shape[0]):
            trainData.set_value(i, 'weeklyAverageSales', weeklySales/count)

In [474]:
weeklySalesAverage()

### Get number of months since last promo2. Does it help: Not sure ###

In [475]:
def intervalToNum(promoInterval):
    promoIntervalNum = list(map(lambda month: month_name_num_dict[month], promoInterval.split(',')))
    return promoIntervalNum

In [476]:
def monthsFromRecentPromo2(row):
    promo2Interval = row['PromoInterval']
    date = row['Date']
    month = date.month
    day = date.day
    monthNumToWord = month_num_name_dict[str(month)]
    promoIntervalNum = intervalToNum(promo2Interval)
    
    if month < promoIntervalNum[0]:
        return (promoIntervalNum[-1] - month) + math.floor(day/daysInMonth[month] + 0.5)
    elif month > promoIntervalNum[-1]:
        return (month - promoIntervalNum[-1]) + math.floor(day/daysInMonth[month] + 0.5)
    else:
        prev = None
        for promoMonthNum in promoIntervalNum:
            curr = promoMonthNum
            if curr > month:
                return (month - prev) + math.floor(day/daysInMonth[month] + 0.5)
            prev = curr

In [477]:
# How many weeks has past since the most recent promo 2
def promo2Month(row):
    promo2Interval = row['PromoInterval']
    date = row['Date']
    month = date.month
    monthNumToWord = month_num_name_dict[str(month)]
    
    if promo2Interval == 0:
        return 0
    elif monthNumToWord in promo2Interval.split(','):
        return 0
    else:
        return monthsFromRecentPromo2(row)

In [478]:
trainData['Promo2Month'] = trainData.apply(lambda row: promo2Week(row), axis=1)
testData['Promo2Month'] = testData.apply(lambda row: promo2Week(row), axis=1)

In [543]:
trainData.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,isYesterdayHoliday,CompetitionAge,HavingPromo2,Promo2Age,DayOfMonth,Month,isStartPromo2,weeklyAverageSales,Promo2Month,holidayBeforeAfter
0,1,5,2015-01-30,5577,616,1,1,0,0,3,...,0.0,78,0,0,30,1,0,4212.142857,0,0
1,1,4,2015-01-29,5171,573,1,1,0,0,3,...,0.0,78,0,0,29,1,0,4212.142857,0,0
2,1,3,2015-01-28,4310,537,1,1,0,0,3,...,0.0,78,0,0,28,1,0,4212.142857,0,0
3,1,2,2015-01-27,4806,532,1,1,0,0,3,...,0.0,77,0,0,27,1,0,4212.142857,0,0
4,1,1,2015-01-26,4781,566,1,1,0,0,3,...,0.0,77,0,0,26,1,0,4212.142857,0,0


In [480]:
testData.head()

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,HasHoliday,isTomorrowHoliday,isYesterdayHoliday,CompetitionAge,HavingPromo2,Promo2Age,DayOfMonth,Month,isStartPromo2,Promo2Month
0,1,5,2015-07-31,555,1,1,0,1,3,1,...,1,0.0,1.0,84,0,0,31,7,0,0
1,1,4,2015-07-30,546,1,1,0,1,3,1,...,1,1.0,1.0,84,0,0,30,7,0,0
2,1,3,2015-07-29,523,1,1,0,1,3,1,...,1,1.0,1.0,84,0,0,29,7,0,0
3,1,2,2015-07-28,560,1,1,0,1,3,1,...,1,1.0,1.0,84,0,0,28,7,0,0
4,1,1,2015-07-27,612,1,1,0,1,3,1,...,1,1.0,0.0,84,0,0,27,7,0,0


In [481]:
def holidayBeforeAfter(row):
    if int(row['HasHoliday']) == 1 or int(row['isYesterdayHoliday']) == 1 or int(row['isTomorrowHoliday']) == 1:
        return 1
    else:
        return 0

### holidayBeforeAfter = 1 day before, during and 1 day after holiday. Does it help: Very slightly###

In [482]:
trainData['holidayBeforeAfter'] = trainData.apply(lambda row: holidayBeforeAfter(row), axis=1)
testData['holidayBeforeAfter'] = testData.apply(lambda row: holidayBeforeAfter(row), axis=1)

In [483]:
trainData.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,isYesterdayHoliday,CompetitionAge,HavingPromo2,Promo2Age,DayOfMonth,Month,isStartPromo2,weeklyAverageSales,Promo2Month,holidayBeforeAfter
0,1,5,2015-01-30,5577,616,1,1,0,0,3,...,0.0,78,0,0,30,1,0,4212.142857,0,0
1,1,4,2015-01-29,5171,573,1,1,0,0,3,...,0.0,78,0,0,29,1,0,4212.142857,0,0
2,1,3,2015-01-28,4310,537,1,1,0,0,3,...,0.0,78,0,0,28,1,0,4212.142857,0,0
3,1,2,2015-01-27,4806,532,1,1,0,0,3,...,0.0,77,0,0,27,1,0,4212.142857,0,0
4,1,1,2015-01-26,4781,566,1,1,0,0,3,...,0.0,77,0,0,26,1,0,4212.142857,0,0


In [484]:
testData.head()

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,isTomorrowHoliday,isYesterdayHoliday,CompetitionAge,HavingPromo2,Promo2Age,DayOfMonth,Month,isStartPromo2,Promo2Month,holidayBeforeAfter
0,1,5,2015-07-31,555,1,1,0,1,3,1,...,0.0,1.0,84,0,0,31,7,0,0,1
1,1,4,2015-07-30,546,1,1,0,1,3,1,...,1.0,1.0,84,0,0,30,7,0,0,1
2,1,3,2015-07-29,523,1,1,0,1,3,1,...,1.0,1.0,84,0,0,29,7,0,0,1
3,1,2,2015-07-28,560,1,1,0,1,3,1,...,1.0,1.0,84,0,0,28,7,0,0,1
4,1,1,2015-07-27,612,1,1,0,1,3,1,...,1.0,0.0,84,0,0,27,7,0,0,1


### Save some data for testing purpose ###

In [503]:
sales = trainData[trainData.Customers != 0].Sales.values
averageWeeklySales = trainData[trainData.Customers != 0].weeklyAverageSales.values

# holidayBeforeAfter
holidayBeforeAfter_Train = trainData[trainData.Customers != 0].holidayBeforeAfter.values
holidayBeforeAfter_Test = testData.holidayBeforeAfter.values

hasHoliday_Train = trainData[trainData.Customers != 0].HasHoliday.values
isTomorrowHoliday_Train = trainData[trainData.Customers != 0].isTomorrowHoliday.values
isYesterdayHoliday_Train = trainData[trainData.Customers != 0].isYesterdayHoliday.values

hasHoliday_Test = testData.HasHoliday.values
isTomorrowHoliday_Test = testData.isTomorrowHoliday.values
isYesterdayHoliday_Test = testData.isYesterdayHoliday.values

# Promo2Month
promo2Month_Train = trainData[trainData.Customers != 0].Promo2Month.values
promo2Month_Test = testData.Promo2Month.values

# isStartPromo2
isStartPromo2_Train = trainData[trainData.Customers != 0].isStartPromo2.values
isStartPromo2_Test = testData.isStartPromo2.values

## Dropping features ##

In [486]:
_trainData = trainData.drop(['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','StateHoliday','SchoolHoliday','Promo2SinceWeek', 'Promo2SinceYear','PromoInterval', 'Promo2', 'Open'], axis=1)
_testData = testData.drop(['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','StateHoliday','SchoolHoliday','Promo2SinceWeek', 'Promo2SinceYear','PromoInterval', 'Promo2', 'Open'], axis=1)
# y = _trainData[_trainData.Customers != 0].Sales.values
# y = _trainData[_trainData.Customers != 0].weeklyAverageSales.values
__trainData = _trainData[_trainData.Customers != 0]
_trainData = _trainData[_trainData.Customers != 0].drop(['Date','Sales', 'Store'], axis=1)
_testData = _testData.drop(['Date','Store'], axis=1)

In [547]:
y = sales

In [544]:
y = averageWeeklySales

In [539]:
y[0:50]

array([ 4212.14285714,  4212.14285714,  4212.14285714,  4212.14285714,
        4212.14285714,  4212.14285714,  3290.42857143,  3290.42857143,
        3290.42857143,  3290.42857143,  3290.42857143,  3290.42857143,
        3980.85714286,  3980.85714286,  3980.85714286,  3980.85714286,
        3980.85714286,  3980.85714286,  4049.42857143,  4049.42857143,
        4049.42857143,  4049.42857143,  4049.42857143,  4049.42857143,
        3871.42857143,  3871.42857143,  3871.42857143,  3871.42857143,
        3871.42857143,  4188.        ,  4188.        ,  4188.        ,
        4188.        ,  6562.42857143,  6562.42857143,  6562.42857143,
        6562.42857143,  6562.42857143,  6562.42857143,  4379.71428571,
        4379.71428571,  4379.71428571,  4379.71428571,  4379.71428571,
        4379.71428571,  5798.57142857,  5798.57142857,  5798.57142857,
        5798.57142857,  5798.57142857])

In [488]:
_trainData = _trainData.drop('weeklyAverageSales', axis=1)

### Don't include DayOfMonth = Improvement in score ###

In [489]:
_trainData = _trainData.drop('DayOfMonth', axis=1)
_testData = _testData.drop('DayOfMonth', axis=1)

### Drop the 3 other holiday indicators ###

In [508]:
_trainData = _trainData.drop(['HasHoliday', 'isTomorrowHoliday', 'isYesterdayHoliday'], axis=1)
_testData = _testData.drop(['HasHoliday', 'isTomorrowHoliday', 'isYesterdayHoliday'], axis=1)

In [504]:
# holidaBeforeAfter
def dropHolidayBeforeAfter():
    _trainData = _trainData.drop('holidayBeforeAfter', axis=1)
    _testData = _testData.drop('holidayBeforeAfter', axis=1)

def undoDropHolidayBeforeAfter():
    _trainData['holidayBeforeAfter'] = holidayBeforeAfter_Train
    _testData['holidayBeforeAfter'] = holidayBeforeAfter_Test

In [ ]:
# undoDropHolidayBeforeAfter()

In [531]:
def dropIsStartPromo2():
    _trainData.drop('isStartPromo2', axis=1, inplace=True)
    _testData.drop('isStartPromo2', axis=1, inplace=True)
def undoDropisStartPromo2():
    _trainData['isStartPromo2'] = isStartPromo2_Train
    _testData['isStartPromo2'] = isStartPromo2_Test

In [532]:
# dropIsStartPromo2()
undoDropisStartPromo2()

In [548]:
_trainData.head()

,DayOfWeek,Customers,Promo,StoreType,Assortment,CompetitionDistance,CompetitionAge,HavingPromo2,Promo2Age,Month,Promo2Month,holidayBeforeAfter,isStartPromo2
0,5,616,1,3,1,0.028061,78,0,0,1,0,0,0
1,4,573,1,3,1,0.028061,78,0,0,1,0,0,0
2,3,537,1,3,1,0.028061,78,0,0,1,0,0,0
3,2,532,1,3,1,0.028061,77,0,0,1,0,0,0
4,1,566,1,3,1,0.028061,77,0,0,1,0,0,0


In [549]:
_testData.head()

,DayOfWeek,Customers,Promo,StoreType,Assortment,CompetitionDistance,CompetitionAge,HavingPromo2,Promo2Age,Month,Promo2Month,holidayBeforeAfter,isStartPromo2
0,5,555,1,3,1,0.028061,84,0,0,7,0,1,0
1,4,546,1,3,1,0.028061,84,0,0,7,0,1,0
2,3,523,1,3,1,0.028061,84,0,0,7,0,1,0
3,2,560,1,3,1,0.028061,84,0,0,7,0,1,0
4,1,612,1,3,1,0.028061,84,0,0,7,0,1,0


In [527]:
X = _trainData
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
rfr = RandomForestRegressor(n_estimators=3)
clf = rfr.fit(X_train, y_train)
print(clf.score(X_val, y_val))
rfr.fit(X,y)
print(rfr.feature_importances_)

0.915505264846
[ 0.00973793  0.77068273  0.02057014  0.05310623  0.01795609  0.06758302
  0.02153848  0.00174163  0.01920249  0.01037825  0.00511354  0.00238947]


In [528]:
results = rfr.predict(_testData.as_matrix())
store_closed_index = np.where(_testData.Customers == 0)

print(store_closed_index)

for val in store_closed_index:
    results[val] = 0
with open('submission.csv', 'w') as csvfile:
    fieldnames = ['Id', 'Sales']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for idx, val in enumerate(results):
        writer.writerow({'Id': index[idx] + 1, 'Sales': val})
    csvfile.close()

(array([    5,    12,    19, ..., 33432, 33439, 33446]),)


How bout some xgboost?

In [545]:
X = _trainData
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
xgb = XGBRegressor(learning_rate=0.3, reg_lambda=0.01, subsample=0.75, colsample_bytree=0.75, max_depth=6)
clf = xgb.fit(X_train, y_train)
print(clf.score(X_val, y_val))
xgb.fit(X,y)
results = xgb.predict(_testData)

0.824522439573


In [546]:
store_closed_index = np.where(_testData.Customers == 0)
print(store_closed_index)

for val in store_closed_index:
    results[val] = 0
with open('submission.csv', 'w') as csvfile:
    fieldnames = ['Id', 'Sales']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for idx, val in enumerate(results):
        writer.writerow({'Id': index[idx] + 1, 'Sales': val})
    csvfile.close()

(array([    5,    12,    19, ..., 33432, 33439, 33446]),)


Previously, we have take account daily trend, but not monthly trends and sales trend

http://yilinwei.com/project/Prediction_of_Rossmann_Store_Sales/Prediction_of_Rossmann_Store_Sales_Report.pdf